<a href="https://colab.research.google.com/github/haayanau/twint/blob/master/transformers_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%time
import tqdm
import transformers
from transformers import BertTokenizer, TFBertModel
import pandas as pd
import spacy
import re
import numpy as np
import tensorflow as tf 
from tensorflow.keras.optimizers import Adam

CPU times: user 29 µs, sys: 4 µs, total: 33 µs
Wall time: 35.8 µs


In [7]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# python -m spacy download en_core_web_sm

In [9]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [10]:
import io

train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [8]:
train = pd.read_csv('C:/Users/16475/PycharmProjects/keras_twitter/haaya_alexei_kaggle/train.csv')
test = pd.read_csv('C:/Users/16475/PycharmProjects/keras_twitter/haaya_alexei_kaggle/test.csv')

FileNotFoundError: ignored

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


nlp = spacy.load("en_core_web_sm")
def preprocessing(text):
  text = text.replace('#','')
  text = decontracted(text)
  text = re.sub('\S*@\S*\s?','',text)
  text = re.sub('http[s]?:(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)

  token=[]
  result=''
  text = re.sub('[^A-z]', ' ',text.lower())
  
  text = nlp(text)
  for t in text:
    if not t.is_stop and len(t)>2:  
      token.append(t.lemma_)
  result = ' '.join([i for i in token])

  return result.strip()

In [ ]:
%%time
train.text = train.text.apply(lambda x : preprocessing(x))
test.text = test.text.apply(lambda x : preprocessing(x))

In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [ ]:
def bert_encode(data,maximum_length) :
  input_ids = []
  attention_masks = []
  

  for i in range(len(data.text)):
      encoded = tokenizer.encode_plus(
        
        data.text[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        
      )
      
      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [ ]:
%%time
train_input_ids,train_attention_masks = bert_encode(train, 60)
test_input_ids,test_attention_masks = bert_encode(test, 60)

In [ ]:
def create_model(bert_model):
  input_ids = tf.keras.Input(shape=(60,),dtype='int32')
  attention_masks = tf.keras.Input(shape=(60,),dtype='int32')
  
  output = bert_model([input_ids,attention_masks])
  output = output[1]
  output = tf.keras.layers.Dense(32,activation='relu')(output)
  output = tf.keras.layers.Dropout(0.2)(output)

  output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
  model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
  model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
# import torch

In [ ]:
%%time
bert_model = TFBertModel.from_pretrained('bert-large-uncased')

In [ ]:
model = create_model(bert_model)
model.summary()

In [ ]:
# filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]
# add_callbacks = callbacks_list to model.fit

In [ ]:
%%time
history = model.fit([train_input_ids,train_attention_masks],train.target,validation_split=0.2, epochs=2,batch_size=10)

In [ ]:
%%time
result = model.predict([test_input_ids,test_attention_masks])
result = np.round(result).astype(int)

In [ ]:
%%time
result = pd.DataFrame(result)
submission = pd.read_csv('C:/Users/16475/PycharmProjects/keras_twitter\haaya_alexei_kaggle/sample_submission.csv')
output = pd.DataFrame({'id':submission.id,'target':result[0]})
output.to_csv('submission.csv',index=False)